Week 1
===
Introduction
---

* notation: 
    * $m$: number of training examples
    * $x, y$: input (feature) and output (target)
    * $(x^{(i)}, y^{(i)})$: the $i$th training example
    
矽谷都用高階語言寫 ML prototype 再 migrate 到 c++ 等

Definition
---

A computer program is said to learn from experience $E$ with respect to some class of tasks $T$ and performance measure $P$, if its performance at tasks in $T$, as measured by $P$, improves with experience $E$. 


Types
---
* supervised
    * regression -- example: given picture, predict age
    * classification 
        * SVM can deal with infinitely many features!
* unsupervised (cluster), examples:
    * organize computing clusters to make grid computing more productive
    * social network
    * market segmentation
    * cocktail party problem: separate 2 speakers' sound; separate music with speaking
    * dataset of patients suffering from heart disease. learn possible clusters to tailor the treatments


Hypothesis and Cost Functions
---
$$
h_{\vec \theta}(y) = \theta_0 + \theta_1 x, \qquad J(\vec \theta) = \frac{1}{2m}\sum_{i=1}^m\left(h_{\vec \theta}(x^{(i)}) - y^{(i)}\right)^2
$$

Gradient Descent
---
update $\theta_0$ and $\theta_1$ simultaneously: 
$$
\vec\theta \leftarrow \vec\theta - \alpha\frac{\partial}{\partial \vec\theta}J(\vec\theta) 
$$

learning rate $\alpha$:  
* too small: slow convergence
* too large: will diverge

Batch Gradient Descent: each step uses all the training examples


Week 2
===

Feature Scaling 
---

* make sure features are on a similar scale: the convergence will be much faster (ziczac convergence path v.s. straight)
* get every feature into [-1, 1] approximately; [0, 3] or [-2, 0.5] are ok, but [-100, 100] is not
* scaling and mean normaliztion (do not apply to constant term $x_0 = 1$)

$$
    \frac{x_i - \mu_i}{\sigma_i}
$$

* depending on the data set, sometimes divide by range (max - min), not sd

Learning Rate
---

* debugging: to check if the learning is converging, plot the $J(\theta)$ against number of iterations; it should decrease abter every iteration
* for different applications, number of iterations before convergence can be very different
* automatic convergence test: declare convergence if $J(\theta)$ decreases by less than $10^{-3}$ in one iteration
    * hard to determine the TOL, so always plot
* if $J(\theta)$ is increasing, or if it's going up and down, most likely the learning rate is too big; use small $\alpha$
* plot the convergence for $\alpha = 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 0.1, \ldots$

Features and Polynomial
---

* create new features like $x_1^2, x_1^3, \sqrt{x_1}, \ldots$
    * feature scaling become important!
    * there are feature selection algorithms that choose for you
    
Normal Equation
---

* $X$: "design matrix" with size (# data points) $\times$ (# features). 
* closed form solution: $\theta = (X^T X)^{-1}X^T y$
* no need for feature scaling, no need to choose $\alpha$, but take $O(n^3)$. 
* can be slower than iterative algorithm when $n > 10^4$
* Octave: ```pint(X'*X)*X'*y```, pinv is pseudo inverse
* when is $X^TX$ non-invertible?
    * linear dependent (redundant) features, like size in sqft and size in $m^2$
    * too many features ($m\leq n$)
* solution to both cases: delete features, or use regularization


Week 3
===

Logistic Regression
---

* classification problem
    * 1 is positive set
    * 0 is negative set, absence of something
* linear regression is too sensitive to outlier to be used as a classification algorithm
* logistic regression:


$$
    h_{\theta}(x) = g(\theta^T x) = \frac{1}{1+e^{-\theta^T x}} = P(y=1~|~x; \theta)
$$


* $x$ is feature, like tumor size

Decision Boundary
---

* $h=0.5$ iff $\theta^T x = 0$; it's a property of hypothesis, not data set
    * if use linear function of features, can only get linear decision boundary
    * To get different shape, use polynomial like $x_1^2, x_1x_2, x_2^2, \ldots$


Cost Function
---

* cost function of logistic regression: cannot use least square; the sigmoid function will make the cost function not convex. Instead, use


$$
J(\theta) = \frac{1}{m}\sum_{i=1}^m \text{Cost}(h_{\theta}(x^{(i)}), y^{(i)})
$$


$$
\text{Cost}(h_{\theta}(x^{(i)}), y^{(i)})=\begin{cases}
        -\log(h_{\theta}(x))&\mbox{ if }y=1\\
        -\log(1 - h_{\theta}(x))&\mbox{ if }y=0
\end{cases}
$$


* There are studies on convexity analysis
* Has property that Cost$\rightarrow\infty$ whenever get wrong prediction
* Simplified cost function: Plug into $J(\theta)$


$$
\text{Cost}(h_{\theta}(x^{(i)}), y^{(i)}) = -y\log(h_{\theta}(x)) - (1-y)\log(1-h_{\theta}(x)) 
$$


* can be derived from MLE
* vectorized: 


$$
h=g(X\theta), \qquad J(\theta)=\frac1m(-y^T\log(h) - (1-y)^T\log(1-h))
$$


* Gradient descent: same $\frac{\partial }{\partial \theta_j}J(\theta)$ formula as linear regression, only $h_\theta$ has different definition
* vectorized: 


$$
\theta \leftarrow \theta - \frac{\alpha}{m}X^T(g(X\theta) - \vec y)
$$


* Feature scaling can also speed up the optimization in logistic regression



Implementation
---

* Optimization algorithm
    * Gradient descent 
    * CG
    * BFGS
        * use an inner loop, the line search algorithm, to pick learning rate automatically 
    * L-BFGS

* Function minimization unconstrained in Octave for
    * @costFunction is function pointer
    * options is a data structure for configuration. 'GraObj' and 'on' means the derivative is provided 
    * initialTheta is the initial guess
    * at least in 2 dimensions
    * help fminunc to read the doc
$$
    J(\theta) = (\theta_1 - 5)^2 + (\theta_2 - 5)^2:
$$

```
function [jVal, gradient] = costFunction(theta)

    jVal = (theta(1)-5)^2 + (theta(1)-5)^2;
    gradient = zeros(2, 1); 
    gradient(1) = 2*(theta(1)-5);
    gradient(2) = 2*(theta(2)-5);
    
options = optimset('GradObj', 'on', 'MaxIter', '100');
initialTheta = zeros(2, 1);
[optTheta, functionVal, exitFlag] = fminunc(@costFunction, initialTheta, options); 
```

Multi-Class
---

* email folder: work, friends, family, hobby...
* one vs all algorithm
* for all $i$, predict $P(~y\in \text{class}~i~|~x; \theta~)$; choose $y\in \text{class }(\text{argmax } P)$

Overfitting and Regularization
---

* too many features, hypothesis fits the training set very well (almost zero cost) but fail to generalize to new examples
* 3 types of fitting result: 
    * underfit (high bias)
    * just right
    * overfit (high variance)
* solution: 
    * reduce number of features
        * manually select which features
        * feature selection algorithms
    * regularization: work well for many slightly useful features


$$
J(\vec \theta) = \frac{1}{2m}\left[\sum_{i=1}^m\left(h_{\vec \theta}(x^{(i)}) - y^{(i)} \right)^2 + \lambda \sum_{i=1}^n \theta_i^2\right]
$$
    

* 1st term: keep good fit
* 2nd term: keep hypothesis simple
* $\lambda$: regularization parameter, control balance
* convention is not to penalize $\theta_0$ term: index starts from 1. For large $\lambda$, $h\approx \theta_0$


Gradient Descent
---
\begin{eqnarray*}
\theta_0 &\leftarrow& \theta_0 - \alpha\frac{1}{m}\sum_{i=1}^m \left(h_{\vec \theta}(x^{(i)}) - y^{(i)} \right)x_0^{(i)}\\
\theta_j &\leftarrow& \theta_j - \alpha\left[\frac{1}{m}\sum_{i=1}^m \left(h_{\vec \theta}(x^{(i)}) - y^{(i)} \right)x_0^{(i)} + \frac{\lambda}{m}\theta_j \right]\\
&& = \theta_j\left(1-\alpha \frac{\lambda}{m}\right) - \alpha\frac{1}{m}\sum_{i=1}^m \left(h_{\vec \theta}(x^{(i)}) - y^{(i)} \right)x_0^{(i)}
\end{eqnarray*}
* $\left(1-\alpha \frac{\lambda}{m}\right) < 1$, say 0.99. slowly makes $\theta_j$ smaller and smaller

Normal Equation with Regularization
---

Solve 
$$
\frac{\partial}{\partial\theta_j} J(\theta) = 0
$$
to get $\theta = [X^TX + \lambda \text{ diag}(0, 1, 1, \ldots, 1)]^{-1} X^T y$
* regularization: if $m<n$ then $X^TX$ non-invertible, but with regularization the matrix is always invertible


Week 4
===

Neural Network Introduction
---

* motivation
    * quadratic features has $O(n^2)$ terms if originally there are $n$ features; cubit features has $O(n^3)$ terms
    * in many practical problems, like computer vision, $n$ is huge
* popular in 80s and early 90s, then diminished in late 90s
* the "one learning algorithm" hypothesis
    * neural rewiring experiment; audio cortex can learn to see
    * "seeing with your tongue", "human sonar", "implanting a 3rd eye", ...

Neuron Model: Logistic Unit
---

* sigmoid (logistic) activation function, $x = (x_0, x_1, x_2, x_3)^T, \theta = (\theta_0, \theta_1, \theta_2, \theta_3)^T$

$$
h_{\vec\theta}(x) = \frac{1}{1+e^{-\theta^T x}}
$$

* $x_0 = 1$, "bias unit"

Neural Network
---

* forward propagation
* input layer, hidden layers, output layer
* $a^{(j)}$ (vector) activation of unit $i$ in layer $j$
* $\theta^{(j)}$ matrix of weights controling function mapping from layer $j$ to layer $j+1$
* if layer $j$ has $s_j$ units than $\theta^{(j)}$ has size $s_{j+1}\times (s_j+1)$
* example: 
    * input layer has 3 units
    * hidden layer has 3 units
    * output layer has 1 unit

\begin{eqnarray*}
a^{(2)} &=& g\left(\theta^{(1)}_{3\times 4} x\right) \\
a^{(2)} &\leftarrow & [1, a^{(2)}]\qquad \% \text{ adding bias $a^{(2)}_0$} \\
h_{\vec\theta} (x) &=& a^{(3)} = g\left(\theta^{(2)}_{1\times 4} a^{(2)}\right) 
\end{eqnarray*}

* without hidden layer: equivalent to logistic regression
* with hidden layer(s): output layer is a logistic regression with features learned from previous layers
* a computer vision example: feature "has a face or not"
* "network architectures": has more than one hidden layers

True Value Tables
---

* how to construct network that computes $x_1$ XOR $x_2$? 
* example: AND
$$
h_{\vec\theta}(x) = g(-30 + 20 x_1 + 20 x_2)
$$


| $x_1$ $x_2$ | 　　　　　　|
|---|-----------------------|
| 0 0 | $g(-30)\approx 0$ |
| 0 1 | $g(-10)\approx 0$ |
| 1 0 | $g(-10)\approx 0$ |
| 1 1 | $g(10)\approx 1$ |

* coefficients determine how strong the pulse is 
* example: OR, $h_{\vec\theta}(x) = g(-10 + 20 x_1 + 20 x_2)$
* example: NOT, $h_{\vec\theta}(x) = g(10 - 20 x_1)$, large negative feature coefficient

| $x_1$ | $h_{\vec\theta}(x)$ |
|---|---|
| 0 | 　　1 |
| 1 | 　　0 |

* can construct some true value tables, e.g. (NOT $x_1$) AND (NOT $x_2$) $=g(-20x_1 - 20x_2 + 10)$
* with multilayers, can construct any logic:
    * example: XNOR (NOT XOR)
    * $x_1$ XNOR $x_2$ = [$x_1$ AND $x_2$] OR [(NOT $x_1$) AND (NOT $x_2$)]$
    * brackets are hidden layer neurons

Multi-Class Classification
---

* outputs are standard basis vectors like $(1, 0, 0, 0)^T$ for the first class, $(0, 1, 0. 0)^T$ for the second, ..., instead of $y\in\{1, 2, 3, 4\}$


Week 5
===

Cost Function
---

* notation
    * $L$: # layers in network
    * $s_l$: # units in layer $l$
* binary classification, $y = 0, 1$
* multi-class classification ($K$ classes), $y = (1, 0, 0, 0)^T, (0, 1, 0, 0)^T, \ldots$
* logistic regression: 


$$
J(\theta) = -\frac{1}{m} \left[\sum_{i=1}^m y^{(i)}\log(h_{\theta}(x^{(i)})) + (1-y^{(i)})\log(1 - h_{\theta}(x^{(i)}))\right] + \frac{\lambda}{2m}\sum_{j=1}^n \theta^2_j
$$


* neural network:


$$
J(\theta) = -\frac{1}{m} \left[\sum_{i=1}^m \sum_{k=1}^K y_k^{(i)}\log(h_{\theta}(x^{(i)})_k) + (1-y^{(i)}_k)\log(1 - h_{\theta}(x^{(i)})_k)\right] + \frac{\lambda}{2m}\sum_{l=1}^{L-1}\sum_{i=1}^{s_{l}}\sum_{j=1}^{s_{l+1}} (\theta_{ji}^{(l)})^2
$$

Backpropagation
---

* forward propagation (4 layers): 

\begin{eqnarray*}
x &\leftarrow& [1, x] \qquad \% \text{ adding bias } \\
a^{(2)} &=& g\left(\theta^{(1)} x\right) \\
a^{(2)} &\leftarrow & [1, a^{(2)}]\qquad \% \text{ adding bias $a^{(2)}_0$} \\
a^{(3)} &=& g\left(\theta^{(2)} a^{(2)}\right) \\
a^{(3)} &\leftarrow & [1, a^{(3)}]\qquad \% \text{ adding bias $a^{(3)}_0$} \\
h_{\vec\theta} (x) = a^{(4)} &=& g\left(\theta^{(1)} a^{(3)}\right) \\
\end{eqnarray*}

* backward propagation: $\delta_j^{(l)}$ is the error of activation $a_j^{(l)}$ at node $j$ in layer $l$

\begin{eqnarray*}
\delta^{(4)} &=& a^{(4)} - y \\
\delta^{(3)} &=& (\theta^{(3)})^T \delta^{(4)}.* g^\prime(\theta^{(2)} a^{(2)}) \\
\delta^{(2)} &=& (\theta^{(2)})^T \delta^{(3)}.* g^\prime(\theta^{(1)} [1, x]) \\
\end{eqnarray*}

* no $\delta^{(1)}$ because first layer is $x$, no error
* if $\lambda = 0$ (no regularization), then the derivative of $J$ is

$$
\frac{\partial}{\partial \theta_{ij}^{(l)}} J(\theta) = a^{(l)}_j\delta_i^{(l+1)}
$$

* it can be shown that $g^\prime(\theta^{(2)} a^{(2)}) = a^{(3)}.*(1-a^{(3)})$
* algorighm: 
    * Training set $\{(x^{(1)}, y^{(1)}), \ldots, (x^{(m)}, y^{(m)})\}$
    * Set $\Delta_{ij}^{(l)} = 0$ for all $l, i, j$
    * For $i=1$ to $m$
        * Set $a^{(1)} = x^{(i)}$
        * Perform forward propagation to compute $a^{(l)}$ for $l = 2, 3, \ldots, L$
        * Using $y^{(i)}$, compute $\delta^{(L)} = a^{(L)} - y^{(i)}$
        * Compute $\delta^{(L-1)}, \delta^{(L-2)}, \ldots, \delta^{(2)}$
        * $\Delta_{ij}^{(l)} = \Delta_{ij}^{(l)} + a_j^{(l)}\delta_{i}^{(l+1)}$
    * $D_{ij}^{(l)} = \frac1m\Delta_{ij}^{(l)} + \lambda \theta_{ij}^{(l)}$ if $j\neq 0$
    * $D_{ij}^{(l)} = \frac1m\Delta_{ij}^{(l)} \qquad$ if $j = 0$
    * $\frac{\partial}{\partial \theta_{ij}^{(l)}} J(\theta) = D_{ij}^{(l)}$

Intuition of Backpropagation
---

* let $z^{(l)} = \theta^{(l-1)}a^{(l-1)}$ (sending $z^{(l)}$ to $g$ will get activation $a^{(l)}$). formally, 

$$
    \delta_j^{(l)} = \frac{\partial}{\partial z_j^{(l)}} \text{cost}(i) = \frac{\partial}{\partial z_j^{(l)}} \left[y^{(i)}\log h_{\theta}(x^{(i)}) + (1-y^{(i)})\log h_{\theta}(x^{(i)})\right]
$$

* how much the cost would change if the intermediate value $z$ changes by a small amount

Unrolling Parameters
---

* $\theta^{(1)}, \theta^{(2)}, \theta^{(3)}, \ldots, D^{(1)}, D^{(2)}, D^{(3)}, \ldots$: matrices
* unroll as a giant (column) vector in octave, to make use of the built-in optimization function: 

```
thetaVector = [Theta1(:); Theta2(:); Theta3(:)]
deltaVector = [D1(:); D2(:); D3(:)]
```

* put it back as matrices: 

```
Theta1 = reshape(thetaVector(1:110), 10, 11)
Theta2 = reshape(thetaVector(111:220), 10, 11)
Theta3 = reshape(thetaVector(221:231), 10, 11)
```

* ```thetaVector(111:220)``` takes the 111th - 220th elements of ```thetaVector```

Gradient Checking
---

* backprop is very complicated; the implementation can be buggy 
* always compare the derivatives obtained by backpropagation with finite difference

$$
\frac{J(\theta+\epsilon) - J(\theta-\epsilon)}{2\epsilon}, \qquad \epsilon = 10^{-4}
$$ 

* $\theta$'s are all vectors obtained by unrolling matrices; compute the partial derivative element by element and put them all together to get the gradient

```
for i = 1:n, 
    thetaPlus = theta;
    thetaPlus(i) = thetaPlus(i) + EPSILON;
    thetaMinus = theta;
    thetaMinus(i) = thetaMinus(i) - EPSILON;
    gradApprox(i) = (J(thetaPlus) - J(thetaMinus))/(2*EPSILON);
end;
```

* check that ```gradApprox``` and ```DVec``` are approximately the same
* implementation: 
    * implement backprop to compute ```Dvec```
    * implement numerical gradient check to compute ```gradApprox```; make sure they give similar values
    * turn off gradient checking; use backprop code for learning
* if forget to turn of gradient checking the training will be very slow

Random Initialization 
---

* "problem of symmetry breaking": if set $\theta_{ji}^{(l)}$ to the same value (say 0) for all $i, j, l$, then all activations in the same layer will be the same, event after many iterations
* initialize each $\theta_{ij}^{(l)}$ to a random value in $[-\epsilon, \epsilon]$

```
Theta1 = rand(10, 11)*(2*INIT_EPSILON) - INIT_EPSILON
Theta1 = rand(1, 11)*(2*INIT_EPSILON) - INIT_EPSILON
```

* ```rand(10, 11)``` gives 10 by 11 random matrix with $U(0, 1)$ distributed elements

Putting it Together
---

* reasonable default: 1 hidden layer, or if >1 hidden layers, have same number of hidden units in every layer (usually the more the better, but also more expensive to train the model)
* training a neural network: 
    1. randomly initialize weights
    * implement forward propagation go get $h_{\theta}(x^{(i)})$ for any $x^{(i)}$
    * implement code to compute cost function $J$
    * implement backprop to compute partial derivatives of $J$
        * for i = 1:m
            * perform forward propagation and backprop using example $(x^{(i)}, y^{(i)})$ (get activations and deltas)
        * there are ways other than for loop to implement backprp, but for the first time just implement the simple for loop version
    * use gradient checking to make sure backprop is implemented correctly; then disable the gradient checking code
    * use gradient descent or advanced optimization method with backprop to minimize $J$
        * $J$ is non-convex, so in theory there is no guarantee of a global minimum, but in practice iterative algorithms work well

